# 农产品定价

## 目标和先决条件

通过这个例子，您将学习如何使用数学优化来解决一个常见但关键的农产品定价问题：确定一个国家乳制品的价格和需求，以最大化这些产品销售带来的总收入。您将学习如何使用Gurobi Python API将此问题建模为二次优化问题，并使用Gurobi优化器求解。

这个模型是H. Paul Williams所著《数学规划中的模型构建》第五版第276-278页和333-335页中的第21个示例。

这个建模示例属于中级水平，我们假设您了解Python并熟悉Gurobi Python API。此外，您应该对构建数学优化模型有一定的了解。


**下载代码库** <br /> 
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

## 问题描述

某国政府想要决定其乳制品：牛奶、黄油和奶酪的价格。所有这些产品都是由该国的原料奶生产运营直接或间接生产的。这些原料奶分为两个主要成分：脂肪和干物质。在扣除用于制造出口产品或农场消费的脂肪和干物质数量后，每年可用的脂肪总量为600,000吨，干物质总量为750,000吨。这些全部可用于生产国内消费的牛奶、黄油和两种奶酪。产品的百分比成分如下表所示：

| 成分 | 脂肪 (%) | 干物质 (%) |
| --- | --- | --- |
| 牛奶 | 4 | 9 |
| 黄油 | 80 | 2 |
| 奶酪1 | 35 | 30 |
| 奶酪2 | 25 | 40 |

下表显示了去年乳制品的国内消费量（需求）和价格：

| 乳制品 | 牛奶 | 黄油 | 奶酪1 | 奶酪2 |
| --- | --- | --- | --- | --- |
| 需求（千吨） | 4.82 | 0.32 | 0.21 | 0.07 |
| 价格（美元/吨） | 297 | 720 | 1050 | 815 |

弹性系数和交叉弹性系数如下表所示：

| 牛奶 | 黄油 | 奶酪1 | 奶酪2 | 奶酪1对奶酪2 | 奶酪2对奶酪1 |
| --- | --- | --- | --- | --- | --- |
| 0.4 | 2.7 | 1.1 | 0.4 | 0.1 | 0.4 |

价格指数不能高于去年。这个约束条件规定，新价格必须使得去年的消费总成本不会增加。
去年的价格指数为1.939（以千美元计）。

目标是确定能够最大化总收入的价格和需求。

## 模型公式

### 集合和索引

$d \in \text{Dairy}=\{\text{milk}, \text{butter}, \text{cheese1}, \text{cheese2} \}$

$c \in \text{Components}=\{\text{fat}, \text{dry_matter} \}$

### 参数

$\text{capacity}_{c} \in \mathbb{R}^+$: 组分$c$的年可用量（千吨）。

$\text{qtyper}_{c,d} \in [0,1]$: 乳制品$d$中组分$c$的百分比。

$\text{consumption}_{d} \in \mathbb{R}^+$: 去年乳制品$d$的国内消费量（千吨）。

$\text{price}_{d} \in \mathbb{R}^+$: 去年乳制品$d$的价格（美元/千吨）。

$\text{elasticity}_{d} \in \mathbb{R}^+$: 去年乳制品$d$的价格弹性。

$\text{elasticity12} \in \mathbb{R}^+$: 去年奶酪1和奶酪2的交叉价格弹性。

$\text{elasticity21} \in \mathbb{R}^+$: 去年奶酪2和奶酪1的交叉价格弹性。

$\text{prcIndex} \in \mathbb{R}^+$: 反映去年消费总成本的价格指数。

### 决策变量

$\text{p}_{d} \in \mathbb{R}^+$: 乳制品$d$的价格（美元/千吨）。

$\text{q}_{d} \in \mathbb{R}^+$: 乳制品$d$的需求（千吨）。

### 约束条件

**产能约束**: 下列约束条件强制执行脂肪和干物质的有限可用性。

\begin{equation}
\sum_{d \in \text{Dairy}}{\text{qtyper}_{c,d}*\text{q}_{d} } \leq \text{capacity}_{c} \quad \forall c \in \text{Components}
\end{equation}


**价格指数**: 此约束条件规定，新价格必须使得去年的消费总成本不会增加。

\begin{equation}
\sum_{d \in \text{Dairy}}{\text{consumption}_{d}*\text{p}_{d} } \leq \text{prcIndex}
\end{equation}

**弹性**: 需求变量$q_{d}$通过价格弹性关系与价格变量$p_{d}$相关。我们用线性关系来近似弹性。

牛奶弹性。
$$
(\text{q}_{milk} - \text{consumption}_{milk})/\text{consumption}_{milk}) = -\text{elasticity}_{milk}*(\text{p}_{milk} - \text{price}_{milk})/\text{price}_{milk})
$$

黄油弹性。
$$
(\text{q}_{butter} - \text{consumption}_{butter})/\text{consumption}_{butter}) = -\text{elasticity}_{butter}*(\text{p}_{butter} - \text{price}_{butter})/\text{price}_{butter})
$$

奶酪1弹性。
$$
(\text{q}_{cheese1} - \text{consumption}_{cheese1})/\text{consumption}_{cheese1}) = -\text{elasticity}_{cheese1}*(\text{p}_{cheese1} - \text{price}_{cheese1})/\text{price}_{cheese1}) 
$$

$$
+ elasticity12*(\text{p}_{cheese2} - \text{price}_{cheese2})/\text{price}_{cheese2})
$$

奶酪2弹性。
$$
(\text{q}_{cheese2} - \text{consumption}_{cheese2})/\text{consumption}_{cheese2}) = -\text{elasticity}_{cheese2}*(\text{p}_{cheese2} - \text{price}_{cheese2})/\text{price}_{cheese2}) 
$$

$$
+ elasticity21*(\text{p}_{cheese1} - \text{price}_{cheese1})/\text{price}_{cheese1}) 
$$

### 目标函数

**收入**: 目标是最大化总收入。

\begin{equation}
\text{最大化} \quad \sum_{d \in \text{Dairy}}{\text{q}_{d}*\text{p}_{d} }
\end{equation}

---
## Python实现

我们导入Gurobi Python模块和其他Python库。

In [ ]:
# %pip install gurobipy

In [1]:
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

## 输入数据

我们定义模型的所有输入数据。

In [2]:
# 乳制品列表

dairy = ['milk', 'butter', 'cheese1', 'cheese2']

components = ['fat', 'dryMatter']


# 创建字典来存储产品的成分百分比

cd, qtyper = gp.multidict({
    ('fat','milk'): 0.04,
    ('fat','butter'): 0.8,
    ('fat','cheese1'): 0.35,
    ('fat','cheese2'): 0.25,
    ('dryMatter','milk'): 0.09,
    ('dryMatter','butter'): 0.02,
    ('dryMatter','cheese1'): 0.3,
    ('dryMatter','cheese2'): 0.4
})

# 创建字典来存储组分的年度可用量（千吨）

components, capacity = gp.multidict({
    ('fat'): 600,
    ('dryMatter'): 750
})

# 创建字典来存储去年的国内消费量和价格

dairy, consumption, price, elasticity = gp.multidict({
    ('milk'): [4.82, 0.297, 0.4],
    ('butter'): [0.32, 0.72, 2.7],
    ('cheese1'): [0.21, 1.05, 1.1],
    ('cheese2'): [0.07, 0.815, 0.4]
})

elasticity12 = 0.1
elasticity21 = 0.4

priceIndex = 1.939

## 模型部署

我们创建一个模型和变量。此模型的决策变量是乳制品的价格和需求。

使用Gurobi求解双线性问题非常简单，只需配置全局参数`nonConvex`，并将此参数设置为2。

In [3]:
model = gp.Model('AgriculturalPricing')

# 设置全局参数
model.params.nonConvex = 2

# 乳制品数量
qvar = model.addVars(dairy, name="qvar")

# 乳制品价格
pvar = model.addVars(dairy, name="pvar")


Set parameter LicenseID to value 2601452
Set parameter NonConvex to value 2


以下约束条件强制执行脂肪和干物质的有限可用性。

In [4]:
# 产能约束

fatCap = model.addConstrs( (gp.quicksum(qtyper[c,d]*qvar[d] for d in dairy) <= capacity[c] for c in components  ), 
                          name='fatCap')

此约束确保新价格必须使得去年的消费总成本不会增加。

In [5]:
# 价格指数约束

priceIndex = model.addConstr( (gp.quicksum(consumption[d]*pvar[d] for d in dairy) <= priceIndex ), name='priceIndex')

需求变量通过价格弹性关系与价格变量相关。我们用线性关系来近似弹性。

In [6]:
# 弹性约束

elasMilk = model.addConstr( (qvar['milk']-consumption['milk'])/consumption['milk']  
                           == -elasticity['milk']*(pvar['milk']-price['milk'])/price['milk'], name='elasMilk')

elasButter = model.addConstr( (qvar['butter']-consumption['butter'])/consumption['butter']  
                           == -elasticity['butter']*(pvar['butter']-price['butter'])/price['butter'], name='elasButter')

elasCheese1 = model.addConstr( (qvar['cheese1']-consumption['cheese1'])/consumption['cheese1']  
                           == -elasticity['cheese1']*(pvar['cheese1']-price['cheese1'])/price['cheese1']
                              +elasticity12*(pvar['cheese2']-price['cheese2'])/price['cheese2'] , name='elasCheese1')

elasCheese2 = model.addConstr( (qvar['cheese2']-consumption['cheese2'])/consumption['cheese2']  
                           == -elasticity['cheese2']*(pvar['cheese2']-price['cheese2'])/price['cheese2']
                              +elasticity21*(pvar['cheese1']-price['cheese1'])/price['cheese1'] , name='elasCheese2')

目标函数是最大化收入。

In [7]:
# 二次目标函数

obj = gp.quicksum(qvar[d]*pvar[d] for d in dairy)

model.setObjective(obj, GRB.MAXIMIZE)

In [8]:
# 验证模型公式

# model.write('AgriculturalPricing.lp')

# 运行优化引擎

model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Non-default parameters:
NonConvex  2

Optimize a model with 7 rows, 8 columns and 22 nonzeros
Model fingerprint: 0xf2586c1c
Model has 4 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-02, 1e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+02]
Presolve removed 2 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 14 rows, 14 columns, 36 nonzeros
Presolved model has 4 bilinear constraint(s)
Variable types: 14 continuous, 0 integer (0 binary)
Found heuristic solution: objective 2.0664080

Root relaxation: objective 2.791205e+00, 10 iterations, 0.00 seconds

## 分析

下表显示了每种乳制品在均衡时的价格（美元/吨）和需求（吨）。产生的总收入为$\$ 2,066,398,260$。

In [9]:
# 输出报告
price_demand = pd.DataFrame(
    {
        "Products": dairy,
        "Price": [f"{round(1000*pvar[d].x):.2f}" for d in dairy],
        "Demand" : [f"{round(1e6*qvar[d].x):.2f}" for d in dairy],
    },
)
price_demand.index=[''] * len(price_demand)
price_demand

,Products,Price,Demand
,milk,322.00,4658728.00
,butter,422.00,677293.00
,cheese1,833.00,265474.00
,cheese2,1111.00,54032.00


---
## 参考文献

H. Paul Williams著《数学规划中的模型构建》第五版。

版权所有 © 2020 Gurobi Optimization, LLC